# 🎬 Director

The `GraphDirector` and `GraphCamera` [behaviors](./Behaviors.ipynb) control and observe
the current contents of the viewport. While the 2D and 3D APIs are very similar, there
are some subtle differences.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import json
from pathlib import Path

import ipyforcegraph.behaviors as B
import ipyforcegraph.graphs as G
import ipywidgets as W
import traitlets as T

In [ ]:
data = json.loads(Path("./datasets/blocks.json").read_text())

## Create Graphs

In [ ]:
c = B.GraphCamera(capture_visible=True)
d = B.GraphDirector()
fg = G.ForceGraph(behaviors=[c, d])
fg.source.nodes, fg.source.links = data["nodes"], data["links"]

In [ ]:
c3 = B.GraphCamera(capture_visible=True)
d3 = B.GraphDirector()
fg3 = G.ForceGraph3D(behaviors=[c3, d3])
fg3.source.nodes, fg3.source.links = data["nodes"], data["links"]

## Create Controls

In [ ]:
xyz = "xyz"

In [ ]:
c_kxyz = {
    x: W.FloatSlider(description=x, min=-2000, max=2000, disabled=True) for x in xyz
}
l_xyz = {
    x: W.FloatSlider(
        description=x, min=-2000, max=2000, disabled=True, layout=dict(display="none")
    )
    for x in xyz
}
c_kxyz["k"] = W.FloatSlider(description="k", min=0, max=10, step=0.001, disabled=True)
c_capture = W.ToggleButton(description="capture", icon="crop")
c_vis = W.IntText(description="visible", disabled=True)

In [ ]:
d_kxyz = {x: W.FloatSlider(description=x, min=-2000, max=2000) for x in xyz}
dl_xyz = {
    x: W.FloatSlider(description=x, min=-2000, max=2000, layout=dict(display="none"))
    for x in xyz
}
d_kxyz["k"] = W.FloatSlider(description="k", min=0, max=5, step=0.001)
durs = ["pan", "zoom"]
d_dur = {dur: W.FloatSlider(description=f"{dur} (s)", max=5) for dur in durs}
btn_action = W.Button(description="action", icon="play", button_style="success")
btn_follow = W.ToggleButton(description="follow", icon="lock", value=True)
sel_user = W.Dropdown(
    description="user",
    options=sorted({d["user"] for d in data["nodes"] if "user" in d}),
)
tmpl_txt = W.Textarea(description="template")
tmpl_nj = B.Nunjucks("")
btn_tmpl = W.ToggleButton(description="use template", icon="filter")
fit_pad = W.FloatSlider(description="padding", min=0, max=200)

In [ ]:
graph_toggle = W.SelectionSlider(description="graph", options={"2D": fg, "3D": fg3})
graph_toggle

In [ ]:
c3.look_at

In [ ]:
def on_zoomed(*_):
    if graph_toggle.value == fg:
        c_kxyz["k"].value = c.zoom
        if btn_follow.value:
            d_kxyz["k"].value = c.zoom
        for x in xyz:
            l_xyz[x].layout.display = "none"
            dl_xyz[x].layout.display = "none"
        for i, v in enumerate(c.center):
            x = xyz[i]
            c_kxyz[x].value = v
            if btn_follow.value:
                d_kxyz[x].value = v
    else:
        c_kxyz["k"].value = 1
        if btn_follow.value:
            d_kxyz["k"].value = 1
        for i, v in enumerate(c3.center):
            x = xyz[i]
            c_kxyz[x].value = v
            if btn_follow.value:
                d_kxyz[x].value = v
        for i, v in enumerate(c3.look_at):
            x = xyz[i]
            l_xyz[x].value = v
            l_xyz[x].layout.display = "flex"
            dl_xyz[x].layout.display = "flex"
            if btn_follow.value:
                dl_xyz[x].value = v


def on_visible(*_):
    c_vis.value = len(c.visible) if graph_toggle.value == fg else len(c3.visible)

In [ ]:
graph_toggle.observe(on_zoomed, "value")
[a_c.observe(on_visible) for a_c in [c, c3]]

T.link((c, "capture_visible"), (c_capture, "value"))
T.link((c3, "capture_visible"), (c_capture, "value"))

c.observe(on_zoomed, ["zoom", "center"])
c3.observe(on_zoomed, ["center", "look_at"])

In [ ]:
def on_direct(*_):
    if graph_toggle.value == fg:
        with d.hold_sync():
            d.zoom = d_kxyz["k"].value
            d.center = [d_kxyz[x].value for x in xyz]
        d.send_state("zoom")
        d.send_state("center")
    else:
        with d3.hold_sync():
            d3.center = [d_kxyz[x].value for x in xyz]
            d3.look_at = [dl_xyz[x].value for x in xyz]
        d3.send_state("look_at")
        d3.send_state("center")


[T.link((d, f"{dur}_duration"), (ds, "value")) for dur, ds in d_dur.items()]
[T.link((d3, f"{dur}_duration"), (ds, "value")) for dur, ds in d_dur.items()]
btn_action.on_click(on_direct)

In [ ]:
def on_tmpl(*_):
    if graph_toggle.value == fg:
        d3.fit_nodes = ""
        if not btn_tmpl.value:
            d.fit_nodes = ""
            return
        d.fit_nodes = tmpl_nj
    else:
        d.fit_nodes = ""
        if not btn_tmpl.value:
            d3.fit_nodes = ""
            return
        d3.fit_nodes = tmpl_nj


T.dlink(
    (sel_user, "value"), (tmpl_txt, "value"), lambda x: "{{ node.user == '%s' }}" % x
)
T.link((d, "fit_padding"), (fit_pad, "value"))
T.link((d3, "fit_padding"), (fit_pad, "value"))
T.dlink((tmpl_txt, "value"), (tmpl_nj, "value"))
btn_tmpl.observe(on_tmpl, "value")

In [ ]:
W.HBox(
    [
        W.VBox(
            [
                graph_toggle,
                W.Label("🎥 Camera"),
                *c_kxyz.values(),
                W.Label("Look At"),
                *l_xyz.values(),
                W.HBox([c_capture, c_vis]),
                W.Label("🎬 Director"),
                W.Tab(
                    [
                        W.VBox(
                            [
                                *d_kxyz.values(),
                                *dl_xyz.values(),
                                *d_dur.values(),
                                W.HBox([btn_follow, btn_action]),
                            ]
                        ),
                        W.VBox([sel_user, tmpl_txt, fit_pad, btn_tmpl]),
                    ],
                    titles=["Bounds", "Template"],
                ),
            ],
            layout=dict(min_width="25em"),
        ),
        fg,
        fg3,
    ],
    layout=dict(height="100%"),
)